# Finetuning LLaMA-3 8B

task: ONR

optim: LoRa

Table of context:
1. Imports and Installs
2. Utils
3. Data Import
4. Model and Trainer Definisions
5. Training and Testing Process

In [1]:
PARAMS = dict({
  'method': 'TL', #TL or DA

  'train_on': ["NB"],
  'test_on': ['NB', 'SB', 'CB'],
  'DA_mixture': [], # NB, SB, CB

  'epochs': 50,
  'LR': 0.5e-4,

  'Fine_Tune': True,
  'Zero-Shot': False})

## Installs and Imports

In [11]:
%%capture
!pip install triton
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [ ]:
%%capture
!pip install xFormers

In [2]:
from tqdm import tqdm
import random
import os
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
import numpy as np
import re
## model

from unsloth import FastLanguageModel
import torch

import pickle
from datasets import Dataset
## training libraries
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Utils

In [3]:
def extract_float(text):
    match = re.search(r"[-+]?\d*\.\d+", text)
    if match:
        return float(match.group(0))
    else:
        return None

def r44(value): return f"{round(value, 3):.3f}"

def r55(value): return f"{round(value, 4):.4f}"

def rrmse(targets, predictions):
    return np.sqrt(((predictions - targets) ** 2).mean())/np.mean(targets)

def metrics(y_test, y_pred, roundd = False):
    rrmse_ = rrmse(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** (1/2)
    r2 = r2_score(y_test, y_pred)
    return rrmse_, rmse, r2

def stid(source_name, target_name):
    return (source_name[0:3] + '-' + target_name[0:3]).lower()

def np_ratio(arr):
    C = Counter(arr)
    return C[0]/C[1]

def evaluator(test_data, model):
    generated_output = []
    real_output = []
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    for testcases in tqdm(test_data):
        inputs = tokenizer(
        [
          alpaca_prompt.format(
              testcases["instruction"], # instruction
              testcases["input"], # input
              "", # output - leave this blank for generation!
          )
      ], return_tensors = "pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

        outputs = tokenizer.batch_decode(outputs)[0]

        outputs = extract_float(outputs.split("### Response:")[1].strip())
        benchmark = extract_float(testcases["output"])

        generated_output.append(outputs)
        real_output.append(benchmark)

    valid_indices = [i for i, output in enumerate(generated_output) if output is not None]
    generated_output_filtered = np.array([generated_output[i] for i in valid_indices])
    real_output_filtered = np.array([real_output[i] for i in valid_indices])
    print('validated generations:', len(generated_output_filtered)/len(test_data))

    rrmse, rmse, r2 = metrics(real_output_filtered, generated_output_filtered)
    # print(rrmse, rmse, r2)
    return rrmse, rmse, r2

def translator(i):
  if i == "SB":
    return "Southern Chicago"
  elif i == "CB":
    return "Central Chicago"
  elif i == "NB":
    return "Northern Chicago"

def get_dataset(trains, add_loc = False):
  datas = []
  for i in trains:
    with open('./serialized_'+i+'_train.pkl', 'rb') as f:
      data = pickle.load(f)
      if add_loc:
        for j in data:
          j['input'] = j['input'] + 'Location: ' + translator(i)
      datas = datas+data

  return datas, Dataset.from_dict({key: [d[key] for d in datas] for key in datas[0]})

def get_test_dataset(trains, add_loc = False):
  datas = []
  for i in trains:
    with open('./serialized_'+i+'_test.pkl', 'rb') as f:
      data = pickle.load(f)
      if add_loc:
        for j in data:
          j['input'] = j['input'] + 'Location: ' + translator(i)
      datas = datas+data

  return datas, Dataset.from_dict({key: [d[key] for d in datas] for key in datas[0]})

def get_dataset_DA(sources, targets, num):
  s, _ = get_dataset(sources, True)
  t, _ = get_dataset(targets, True)
  random_indices = random.sample(range(len(t)), num)

  for idx in random_indices:
    s.append(t[idx])
  random.shuffle(s)
  datas = s
  return datas, Dataset.from_dict({key: [d[key] for d in datas] for key in datas[0]})

results = []
def logger(source, target, rrmse, rmse, r2, model = 'LLaMA3-8B'):
    results.append({
        'model': model,
        'source': source,
        'target': target,
        'rrmse': rrmse,
        'rmse': rmse,
        'r2': r2,
    })

## Model

In [4]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Import

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.\
                   Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{} Is the water [safe] or [unsafe], select one based on the information.

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [7]:
if PARAMS['method'] == 'TL':
  print('TL train data')
  train_dict, train_dataset = get_dataset(PARAMS['train_on'])
  dataset = train_dataset.map(formatting_prompts_func, batched = True,)
  print('train data', PARAMS['train_on'], len(train_dict))

TL train data


Map:   0%|          | 0/2464 [00:00<?, ? examples/s]

train data ['NB'] 2464


In [8]:
if PARAMS['method'] == 'DA':
  print('DA train data')
  train_dict, train_dataset = get_dataset_DA(PARAMS['train_on'], PARAMS['DA_mixture'], 500)
  dataset = train_dataset.map(formatting_prompts_func, batched = True,)
  print('train data', PARAMS['train_on'], len(train_dict))

In [9]:
test_dict = {}
for i in PARAMS['test_on']:
  if PARAMS['method'] == 'DA':
    test_dict_, test_dataset = get_test_dataset([i], add_loc=True)
  else:
    test_dict_, test_dataset = get_test_dataset([i])
  test_dict[i] = test_dict_

  print(i, 'test data', len(test_dict_))

NB test data 1316
SB test data 1525
CB test data 869


## Train and Evaluate

In [10]:
if PARAMS['Zero-Shot']:
  for i in test_dict.keys():
    rrmse, rmse, r2 = evaluator(test_dict[i], model)
    logger(PARAMS['train_on'], i, rrmse, rmse, r2, model = 'LLaMA3-8B, 0-Shot')
    print(results[-1])

In [11]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = PARAMS['epochs'],
        learning_rate = PARAMS['LR'],
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.02,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2464 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
if PARAMS['Fine_Tune']:
  trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,464 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.380000
2,2.377800
3,2.382500
4,2.359400
5,2.326400


In [ ]:
if PARAMS['Fine_Tune']:
  for i in test_dict.keys():
    rrmse, rmse, r2 = evaluator(test_dict[i], model)
    logger(PARAMS['train_on'], i, rrmse, rmse, r2, model = 'LLaMA3-8B, FT on '+PARAMS['train_on'])
    print(results[-1])

In [ ]:
for i in results:
  print(i)
if len(results) == 0:
  print('Nothing has been done.')